In [1]:
import base64
from Crypto.Cipher import AES
from Crypto import Random

BS = 16

def pad(s):
    result =  s + (BS - len(s.encode("utf-8")) % BS) * chr(BS - len(s.encode("utf-8")) % BS)
    return result

def unpad(s):
    return s[:-ord(s[len(s)-1:])]

class AESCipher(object):
    def __init__(self, key):
        self.key = key

    def encrypt(self, raw):
        raw = pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return base64.b64encode(iv + cipher.encrypt(raw))

    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(enc[16:]))
    
secret = 'vQ4UZ8toFy2pUMXxeKn3F28vZREJPJ63'
license_file_secret = 'oLMqbk4KpJ2kmVYGBcm3WZ6wCg2LFDNL'

aes_cipher = AESCipher(secret)
lic_cipher = AESCipher(license_file_secret)

In [2]:
import pandas as pd

def decrypt_ea_log(source_filename):
    df = pd.read_csv(source_filename, delimiter='\t', header=None)
    df.columns = ['eng', 'arb', 'is_from_dict']
    
    for index, row in df.iterrows():
        try:
            row[0] = aes_cipher.decrypt(row[0]).decode()
            row[1] = aes_cipher.decrypt(row[1]).decode()
            row[2] = aes_cipher.decrypt(row[2]).decode()
        except:
            print("index: {i}".format(i=index))
            print("row: {a}\t{b}\t{c}".format(a=row[0], b=row[1], c=row[2]))
            raise
    return df

def decrypt_dict(source_filename):
    df = pd.read_csv(source_filename, delimiter='\t', index_col=0)
    
    for index, row in df.iterrows():
        try:
            row['eng'] = aes_cipher.decrypt(row['eng']).decode("utf-8")
            row['arb'] = aes_cipher.decrypt(row['arb']).decode("utf-8")
        except:
            print("index: {i}".format(i=index))
            print("row: {a}\t{b}".format(a=row[0], b=row[1]))
            raise
    return df
    
def encrypt_dict(source_filename):
    df = pd.read_csv(source_filename, delimiter='\t', index_col=0, usecols=[0,1,2])
    for index, row in df.iterrows():
        try:
            row['eng'] = aes_cipher.encrypt(row['eng']).decode()
            row['arb'] = aes_cipher.encrypt(row['arb']).decode("utf-8")
        except:
            print("index: {i}".format(i=index))
            print("row: {a}\t{b}".format(a=row[0], b=row[1]))
            raise
    return df

base_dir = "/home/jupyter/notebooks/PoC/data-preparation/output/transliteration_datasets/"
    

In [11]:
ea_df = decrypt_ea_log(base_dir + 'ea_2.log')
ea_df.to_csv( base_dir + 'ea_2_decrypted.tsv', sep='\t')

In [4]:
ed_df_encrypted = encrypt_dict(base_dir +'eng_positive_trans.tsv')
ed_df_encrypted.to_csv(base_dir + 'ea_dict.tsv', sep='\t')

In [ ]:
ed_df_decrypted = decrypt_dict(base_dir + 'ea_dict.tsv')
ed_df_decrypted.to_csv(base_dir + 'ea_dict_plain.tsv', sep='\t')

In [ ]:
#create license file
from time import gmtime, strftime
from datetime import datetime

#strftime("%Y-%m-%d %H:%M:%S", gmtime())
datetime_object = datetime.strptime("2018-03-01 18:18:08", "%Y-%m-%d %H:%M:%S")
datetime.strftime(datetime_object, "%Y-%m-%d %H:%M:%S")

In [ ]:
datetime.strftime(datetime_object, "%Y-%m-%d %H:%M:%S")

In [6]:
l = list(secret)

In [10]:
''.join(l)

'vQ4UZ8toFy2pUMXxeKn3F28vZREJPJ63'